# Init.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import calendar 

# Load Data

> Read data realtime and save to local

In [ ]:
# # Data Source: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

# data_name_june = "yellow_tripdata_2025-06.parquet"
# data_name_july = "yellow_tripdata_2025-07.parquet"

# raw_data_url_june = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-06.parquet"
# raw_data_url_july = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-07.parquet"

# df_june = pd.read_parquet(raw_data_url_june)
# df_july = pd.read_parquet(raw_data_url_july)

# df_june.to_parquet(f"./data/{data_name_june}")
# df_july.to_parquet(f"./data/{data_name_july}")

In [ ]:
# Read Data
taxi_zone = pd.read_csv('./data/taxi_zone_lookup.csv')
trip_data_06 = pd.read_parquet('./data/yellow_tripdata_2025-06.parquet')

In [ ]:
# Preview Data
taxi_zone

In [ ]:
# Preview Data
display(trip_data_06.head())
display(trip_data_06.tail())

# Data Understanding  

## Data Inspection

### Taxi Zone

In [ ]:
taxi_zone.info()

In [ ]:
trip_data_06.info()

In [ ]:
# Check Missing Value (Taxi Zone)
taxi_zone.isnull().sum()

In [ ]:
# Preview Missing Value (Taxi Zone)
taxi_zone[taxi_zone["service_zone"].isnull()]

In [ ]:
# Check Duplicates (Taxi Zone)
taxi_zone.duplicated().sum()

In [ ]:
# Check Missing Value (Taxi Zone)
taxi_zone.describe(include='all')

> Something weird with zone?  
> Zone mode is `Governor's Island/Ellis Island/Liberty Island` w/ freq. 3

In [ ]:
# Preview taxi zone with same name but different Loc.ID
taxi_zone[taxi_zone["Zone"]=="Governor's Island/Ellis Island/Liberty Island"]

### Trip Data (06)

> **Univariate**

In [ ]:
trip_data_06.describe()

> What are looks suspicious?

- `VendorID` & `RatecodeID`: Some categories not listed in metadata
- `passenger_count`: 0?
- `trip_distance`:  max value look like doesn't make sense
- `fare_amount`: have minus & max value look like doesn't make sense
- `tip_amount`: have minus & max value look like doesn't make sense
- `.describe()` -> float / numerical not ID

#### `fare_amount`  


> **Tips**: _Start with Target Variable_

In [ ]:
# Check outlier / distribution / unreasonable value
trip_data_06[["fare_amount"]].describe()

In [ ]:
# Easy way to check outlier / distribution
trip_data_06[["fare_amount"]].boxplot(vert=False);
plt.show()

In [ ]:
# Other way to check outlier / distribution
trip_data_06[["fare_amount"]].hist(bins=100);
plt.show()

In [ ]:
# Preview minus fare_amount
trip_data_06[trip_data_06.fare_amount < 0].fare_amount.describe()

In [ ]:
# Preview very high fare_amount
trip_data_06[
    trip_data_06.fare_amount > trip_data_06.fare_amount.quantile(.999999) # 99.9999% data
]#.fare_amount.describe()

> Notes:  

- Fare amount have negative value (e.g. -3.5, -5, -10, etc). is it data entry error, refund, discount, or what?
- Very high fare amount (e.g. > 500 or more than 99.9999% data) can be outlier or special case (e.g. long distance, luxury car, etc)
- Need to check with other columns (e.g. payment_type, extra, mta_tax, tip_amount, tolls_amount, total_amount)
- Need to check with trip_distance = 0

#### `VendorID`

In [ ]:
# Check Unique Value
trip_data_06.VendorID.unique()

In [ ]:
# Check Freq. / Total Data for each VendorID (manual way)
for vendor_id in trip_data_06.VendorID.unique():
    vendor_id_count = trip_data_06[trip_data_06.VendorID == vendor_id].shape[0]
    print(f"Total Vendor ID {vendor_id} in data: {vendor_id_count} rows")

In [ ]:
# Check Freq. / Total Data for each VendorID (better way)
tmp = trip_data_06.copy() # NOTE: This is just a safe way
tmp["count"] = 1 # Dummy value
tmp.groupby(["VendorID"]).count()["count"]

In [ ]:
# Check Freq. / Total Data for each VendorID (better way)
trip_data_06.VendorID.value_counts(
    # normalize=True
) # * 100

> Question: What should we do with VendorID except 1 and 2?

#### `trip_distance`

In [ ]:
# Preview trip_distance statistical description again
trip_data_06.trip_distance.describe()

In [ ]:
# Filter very long trip_distance
very_long_trip = trip_data_06[
    trip_data_06.trip_distance > trip_data_06.trip_distance.quantile(.99995) # 99.9999% data
].copy()

# Plot very long trip_distance
very_long_trip.trip_distance.hist(bins=50);
plt.title('Trip Distance > 99.995% Data');
plt.xlabel('Trip Distance (miles)');
plt.ylabel('Frequency');
plt.show();
very_long_trip.trip_distance.describe()

> **Multivariate Analysis** 

In [ ]:
# Check fare_amount correlation with trip_distance
display(trip_data_06[["fare_amount", "trip_distance"]].corr())
sns.scatterplot(
    data=trip_data_06, 
    x="trip_distance", 
    y="fare_amount"
)\
.set_title("Fare Amount vs Trip Distance");
plt.show()

#### `passenger_count`

In [ ]:
# Preview passenger_count statistical description
trip_data_06.passenger_count.describe()

In [ ]:
trip_data_06.passenger_count.isnull().sum() 

In [ ]:
# Check Passenger Count Distribution
(trip_data_06\
    .value_counts(
        "passenger_count", 
        normalize=True
    ) *100
).plot(kind="bar");
plt.show();

In [ ]:
# Better visualization
(trip_data_06.value_counts("passenger_count", normalize=True) *100).sort_index().plot(kind="bar", rot=0);
plt.xlabel("Passenger Count");
plt.ylabel("Percentage (%)");
plt.title("Distribution of Passenger Count");
plt.show();

In [ ]:
# Average fare_amount by passenger_count.
# Alternative 1: by grouping
trip_data_06[[
    "fare_amount",
    "passenger_count"
]].groupby("passenger_count").mean()

In [ ]:
# Average fare_amount by passenger_count.
# Alternative 2: by pivot_table
# NOTE: uncomment the plot code to visualize
trip_data_06.pivot_table(
    values="fare_amount",
    index="passenger_count",
    aggfunc="mean"
)#\
#.plot(kind="bar");
#plt.show()

In [ ]:
# For better visualization
# Filter data, remove NaN, convert to int, than plot
tmp_df = trip_data_06[["fare_amount", "passenger_count"]].copy()
tmp_df.dropna(subset=["passenger_count"], inplace=True)
tmp_df.passenger_count = tmp_df.passenger_count.astype(int)
tmp_df.pivot_table(
    values="fare_amount",
    index="passenger_count",
    aggfunc="mean"
)\
.plot(kind="bar", rot=0);
plt.xlabel("Passenger Count");
plt.ylabel("Average Fare Amount ($)");
plt.title("Average Fare Amount by Passenger Count");
plt.show()

#### `<Other Features>`

In [ ]:
# NOTE: please explore it yourself ;)


#### **Checkpoints**:

What you have learned so far (in terms of idea): 
1. Understand the data (data types, missing values, duplicates, basic statistics)
2. Identify potential issues (outliers, unreasonable values)
3. Works with numerical and categorical variables

What you have learned so far (in terms of technical): 
1. Simple visualization (boxplot, histogram)
2. Statistical summary / descriptive for numerical (mean, median, std, min, max, quantiles)
3. Statistical summary / descriptive for categorical (mode, freq., n_unique)
4. Filtering Dataframe
5. Univariate analysis (focus on one variable at a time)
6. Multivariate analysis (focus on more than one variable)
7. Data aggregation
5. Create visualization (basic and better)

# Pre-processing

## Combine Data

In [ ]:
# Concat two months data
trip_data_07 = pd.read_parquet('./data/yellow_tripdata_2025-07.parquet')
trip_data = pd.concat([trip_data_06, trip_data_07])

In [ ]:
# Merge with taxi zone data
trip_data = trip_data.merge(taxi_zone, left_on='PULocationID', right_on='LocationID', how='left')
trip_data = trip_data.merge(taxi_zone, left_on='DOLocationID', right_on='LocationID', how='left', suffixes=('_PULocation', '_DOLocation'))

In [ ]:
display(trip_data.head(2))
display(trip_data.tail(2))

### Tips & Trick: **Optimization Dtype**

In [ ]:
# Store original data (for comparation purpose)
trip_data_ori = trip_data.copy()

# Columns that should be int
should_be_int = [
    "VendorID",
    "passenger_count",
    "RatecodeID",
    "payment_type",
    "PULocationID",
    "DOLocationID",
]

# Convert to int
for col in should_be_int:
    trip_data[col] = trip_data[col].fillna(-1).astype(int) # Pandas nullable integer type

# Optimize dataset (in term of memory usage)
for col in trip_data.select_dtypes("float64").columns:
    trip_data[col] = pd.to_numeric(trip_data[col], downcast="float")
for col in trip_data.select_dtypes("int32").columns:
    trip_data[col] = pd.to_numeric(trip_data[col], downcast="integer")

In [ ]:
trip_data_ori.info()

In [ ]:
trip_data.info()

## Feature Engineering

In [ ]:
# Create derrived features
trip_data["pickup_hour"] =  trip_data["tpep_pickup_datetime"].dt.hour
trip_data["pickup_day"] =  trip_data["tpep_pickup_datetime"].dt.weekday

In [ ]:
# Average fare_amount by pickup_hour
trip_data.pivot_table(
    values="fare_amount",
    index="pickup_hour",
    aggfunc="mean"
).plot(kind="bar", rot=0, figsize=(10,3), legend=False);
plt.xlabel("Hour of Day");
plt.ylabel("Average Fare Amount ($)");
plt.title("Average Fare Amount by Hour of Day");
plt.show()

# Average fare_amount by pickup_day
trip_data.pivot_table(
    values="fare_amount",
    index="pickup_day",
    aggfunc="mean"
).plot(kind="bar", rot=0, figsize=(10,3), legend=False);
plt.xlabel("Day of Week");
plt.ylabel("Average Fare Amount ($)");
plt.title("Average Fare Amount by  Day of Week");
plt.show()

In [ ]:
# Create a copy of selected columns to analyze time-related patterns
time_perspective = trip_data[["tpep_pickup_datetime", "fare_amount"]].copy()

# Extract hour of day and weekday number (0 = Monday, 6 = Sunday)
time_perspective["hour"] = time_perspective["tpep_pickup_datetime"].dt.hour
time_perspective["weekday"] = time_perspective["tpep_pickup_datetime"].dt.weekday
time_perspective["date"] = time_perspective["tpep_pickup_datetime"].dt.date

# Map weekday numbers to weekday names (e.g., 0 → Monday)
weekday_map = {i: calendar.day_name[i] for i in range(7)}
time_perspective["weekday_name"] = time_perspective["weekday"].map(weekday_map)

# ==== 1) Average fare amount by day of week and hour ====

# Create a pivot table for the average fare
heatmap_fare = time_perspective.pivot_table(
    index="weekday_name",
    columns="hour",
    values="fare_amount",
    aggfunc="mean"
)

# Sort rows from Monday to Sunday
heatmap_fare = heatmap_fare.reindex([calendar.day_name[i] for i in range(7)])

# Plot heatmap of average fares
plt.figure(figsize=(12, 4))
ax = sns.heatmap(
    heatmap_fare,
    cmap="RdYlGn_r",
    cbar_kws={"label": "Average Fare Amount (USD)"}
)

# Add labels and title
ax.set_xlabel("Hour of Day", fontsize=12)
ax.set_ylabel("Day of Week", fontsize=12)
plt.xticks(rotation=0)
plt.yticks(rotation=0)
plt.title("Average Trip Fare by Day of Week and Hour", fontsize=16, weight="bold")
plt.tight_layout()
plt.show()

# ==== 2) Trip counts by day of week and hour ====

# Create a pivot table for the number of trips
heatmap_counts = time_perspective.pivot_table(
    index="weekday_name",
    columns="hour",
    values="date",
    aggfunc="count"
)

# Sort rows from Monday to Sunday
heatmap_counts = heatmap_counts.reindex([calendar.day_name[i] for i in range(7)])

# Plot heatmap of trip counts
plt.figure(figsize=(12, 4))
ax = sns.heatmap(
    heatmap_counts,
    cmap="RdYlGn_r",
    cbar_kws={"label": "Number of Trips"}
)

# Add labels and title
ax.set_xlabel("Hour of Day", fontsize=12)
ax.set_ylabel("Day of Week", fontsize=12)
plt.xticks(rotation=0)
plt.yticks(rotation=0)
plt.title("Trip Counts by Day of Week and Hour", fontsize=16, weight="bold")
plt.tight_layout()
plt.show()


## Feature Selection

In [ ]:
target = "fare_amount"
features = [
    "trip_distance",
    "VendorID",
    "passenger_count",
    "pickup_hour",
    "pickup_day",

]

trip_data_clean = trip_data[features + [target]].copy()
trip_data_clean.info()

In [ ]:
# Optimize dataset (in term of memory usage/size)
trip_data_clean.VendorID = trip_data_clean.VendorID.astype('uint8')
trip_data_clean.pickup_hour = trip_data_clean.pickup_hour.astype('uint8')
trip_data_clean.pickup_day = trip_data_clean.pickup_day.astype('uint8')
trip_data_clean.fare_amount = trip_data_clean.fare_amount.astype('float32')
trip_data_clean.trip_distance = trip_data_clean.trip_distance.astype('float32')
trip_data_clean.info()

> Notes: _We shrank the data size by over **94%**, dropping **from ~1.6 GB to only ~94 MB**._

## Data Cleaning

### Handle Duplicates (if any)

In [ ]:
# Check duplicates
total_duplicates_data = trip_data_clean.duplicated().sum()
print(f"Total Duplicates Data: {total_duplicates_data.sum()} rows / {total_duplicates_data / trip_data_clean.shape[0] * 100:.2f}%")

# Preview duplicates
trip_data_clean[trip_data_clean.duplicated(keep=False)].sort_values(by=target)

In [ ]:
# Keep all if want to analyze duplicates later
dup_data = trip_data_clean.duplicated(keep=False)

# Drop duplicates
trip_data_clean = trip_data_clean.drop_duplicates(keep="first")
trip_data_clean.reset_index(drop=True, inplace=True)
trip_data_clean.info()

### Handling Missing Value (if any)

In [ ]:
# Check missing value
trip_data_clean.isnull().sum()

> Notes: remember that we change / fillna by -1 earlier

In [ ]:
# Review data with passenger_count missing value vs complete
display(trip_data_clean[trip_data_clean.passenger_count.isnull()].describe()) 
display(trip_data_clean[~trip_data_clean.passenger_count.isnull()].describe()) 

In [ ]:
# Check Mode for passenger_count
(trip_data_clean.passenger_count.value_counts(normalize=True)*100).sort_index().plot(kind="bar", rot=0, figsize=(10,3));
plt.xlabel("Passenger Count");
plt.ylabel("Percentage (%)");
plt.title("Distribution of Passenger Count");
plt.show()

In [ ]:
# Fillna with mode
mode_passenger_count = 1
trip_data_clean.passenger_count = trip_data_clean.passenger_count.replace(-1, mode_passenger_count)

In [ ]:
# Check Mode for passenger_count
(trip_data_clean.passenger_count.value_counts(normalize=True)*100).sort_index().plot(kind="bar", rot=0, figsize=(10,3));
plt.xlabel("Passenger Count");
plt.ylabel("Percentage (%)");
plt.title("Distribution of Passenger Count (After Fillna / Replacing -1 with Mode)");
plt.show()

### Handling Outlier (if any)

In [ ]:
# trip_data_clean.describe()# .corr()[[target]].sort_values(by=[target], ascending=False).style.background_gradient(cmap='bwr')
numerical_features = [
    'trip_distance',
    'passenger_count',
    'fare_amount',
]
categorical_features = [
    'VendorID',
    'pickup_hour',
    'pickup_day',
]
for col in numerical_features:
    print(f"Statistical Description of `{col}`:")
    print(trip_data_clean[col].describe().round(2))
    trip_data_clean[[col]].boxplot(rot=0, figsize=(8,3), vert=False);
    plt.title(f"Boxplot of `{col}`");
    plt.show()
    print("\n")

In [ ]:
# Option 1: Remove outliers using IQR rule
def remove_outliers_iqr(df, columns, multiplier=1.5):
    """
    Remove outliers from selected numeric columns using the IQR rule.

    Parameters
    ----------
    df : pandas.DataFrame
        The input DataFrame.
    columns : list of str
        List of column names where you want to remove outliers.
    multiplier : float, optional, default=1.5
        The IQR multiplier (1.5 is common; use 3.0 for more relaxed filtering).

    Returns
    -------
    pandas.DataFrame
        A copy of the DataFrame with outliers removed.
    """
    clean_df = df.copy()

    for col in columns:
        if col not in clean_df.columns:
            raise KeyError(f"Column '{col}' not found in DataFrame.")

        if not pd.api.types.is_numeric_dtype(clean_df[col]):
            raise TypeError(f"Column '{col}' is not numeric.")

        q1 = clean_df[col].quantile(0.25)
        q3 = clean_df[col].quantile(0.75)
        iqr = q3 - q1
        lower = q1 - multiplier * iqr
        upper = q3 + multiplier * iqr

        clean_df = clean_df[(clean_df[col] >= lower) & (clean_df[col] <= upper)]

    return clean_df

filtered_with_iqr = remove_outliers_iqr(trip_data_clean, ["fare_amount", "trip_distance"])
filtered_with_iqr.reset_index(drop=True, inplace=True)

# Option 2: Remove outliers using Percentile method
MAX_PERCENTILE = .9999
trip_data_clean.fare_amount.quantile(MAX_PERCENTILE), trip_data_clean.trip_distance.quantile(MAX_PERCENTILE)
filtered_with_percentile = trip_data_clean[
    (trip_data_clean.fare_amount <= trip_data_clean.fare_amount.quantile(MAX_PERCENTILE)) &
    (trip_data_clean.trip_distance <= trip_data_clean.trip_distance.quantile(MAX_PERCENTILE))
]
filtered_with_percentile.reset_index(drop=True, inplace=True)

# Option 3: Manual filtering
filtered_manual = trip_data_clean[
    (trip_data_clean.fare_amount > 0) & 
    (trip_data_clean.fare_amount < 500) &
    (trip_data_clean.trip_distance > 0) &
    (trip_data_clean.trip_distance < 250)
]
filtered_manual.reset_index(drop=True, inplace=True)

# Preview filtered data
for col in ["fare_amount", "trip_distance"]:
    filtered_with_percentile[[col]].boxplot(rot=0, figsize=(8,3), vert=False);
    plt.title(f"Boxplot of `{col}` after removing outliers using Percentile method");
    plt.show()
    print("\n")
print(f"Reduced {trip_data_clean.shape[0]-filtered_with_percentile.shape[0]} rows to ",
      f"{(trip_data_clean.shape[0]-filtered_with_percentile.shape[0])/trip_data_clean.shape[0]*100:.2f}% rows after removing outliers using Percentile method.")

for col in ["fare_amount", "trip_distance"]:
    filtered_with_iqr[[col]].boxplot(rot=0, figsize=(8,3), vert=False);
    plt.title(f"Boxplot of `{col}` after removing outliers using IQR method");
    plt.show()
    print("\n")
print(f"Reduced {trip_data_clean.shape[0]-filtered_with_iqr.shape[0]} rows to ",
      f"{(trip_data_clean.shape[0]-filtered_with_iqr.shape[0])/trip_data_clean.shape[0]*100:.2f}% rows after removing outliers using IQR method.")

for col in ["fare_amount", "trip_distance"]:
    filtered_manual[[col]].boxplot(rot=0, figsize=(8,3), vert=False);
    plt.title(f"Boxplot of `{col}` after removing outliers using Manual Filtering");
    plt.show()
    print("\n")
print(f"Reduced {trip_data_clean.shape[0]-filtered_manual.shape[0]} rows to ",
      f"{(trip_data_clean.shape[0]-filtered_manual.shape[0])/trip_data_clean.shape[0]*100:.2f}% rows after removing outliers using Manual Filtering.")


In [ ]:
# Compare correlation after outlier removal
display(filtered_with_iqr.corr()[[target]].drop(target).sort_values(by=[target], ascending=False).style.background_gradient(cmap='bwr'))
display(filtered_with_percentile.corr()[[target]].drop(target).sort_values(by=[target], ascending=False).style.background_gradient(cmap='bwr'))
display(filtered_manual.corr()[[target]].drop(target).sort_values(by=[target], ascending=False).style.background_gradient(cmap='bwr'))

In [ ]:
# Check fare_amount correlation with trip_distance (after outlier removal) with viz.
sns.scatterplot(
    data=filtered_manual, 
    x="trip_distance", 
    y="fare_amount"
)\
.set_title("Fare Amount vs Trip Distance");

plt.show()

## Preview Clean Data

In [ ]:
filtered_manual.sample(10)

In [ ]:
filtered_manual.info()

In [ ]:
filtered_manual.describe()

#### **Checkpoints**:

What you have learned so far (in terms of idea): 
1. Handling duplicate value
2. Handling missing value
3. Handling outlier

# Export Data

In [ ]:
filtered_manual.to_parquet('./data/trip_data_clean.parquet', index=False)
filtered_manual.to_csv('./data/trip_data_clean.csv', index=False)